# Amazon Bedrock boto3 のセットアップ

> *このノートブックは Amazon SageMaker Studio の **`Data Science 3.0`** kernel で実行してください*

---

このノートブックでは、[`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) を使って、 [Amazon Bedrock](https://aws.amazon.com/bedrock/) の基盤モデルを使用する方法を確認します.

---

## 前提

このセクションのセルを実行して、このワークショップのノートブックで必要なパッケージをインストールしてください。  
⚠️ pip の依存関係のエラーが表示されますが、これらのエラーは安全に無視できます。⚠️

エラーは無視してください: pip の依存関係リゾルバーは、現在インストールされているすべてのパッケージを考慮していません。この動作が次の依存関係の競合の原因です。

In [ ]:
%pip install --upgrade pip
%pip install --no-build-isolation --force-reinstall "boto3==1.34.64" \
    "awscli==1.32.64" \
    "botocore==1.34.64"
%pip install "langchain==0.1.11" \
    "transformers" \
    "faiss-cpu" \
    "pypdf" \
    datasets
%pip install anthropic
%pip install google-search-results
%pip install SQLAlchemy
%pip install flask-sqlalchemy --user

In [ ]:
%pip install anthropic==0.20.0

In [ ]:
%pip install pydantic

### カーネル再起動
上記セルを実行後、メニューバーの `Kernel` から、`Restart Kernel...` を選択しカーネルを再起動します。

---

## 一般的な推論パラメータの定義

### ランダム性と多様性

基盤モデルは、応答のランダム性と多様性を制御するために、次のパラメータをサポートしています。

**Temperature** - 大規模言語モデルは、シーケンス内の単語を構築するために確率を使用します。任意の次の単語について、シーケンス内の次の単語のオプションの確率分布があります。`Temperature` を 0 に近づけると、モデルはより高い確率の単語を選択する傾向があります。`Temperature` を 1 に近づけると、モデルは低い確率の単語を選択する可能性があります。

技術的には、`Temperature` は次のトークンの確率密度関数を調整しします。このパラメータは、密度関数の曲線を深くまたは浅くすることができます。値が低いと、曲線はより急峻になり、確定的な応答が増えます。値が高いと、曲線はより平坦になり、ランダムな応答が増えます。

**Top K** - `Temperature` は可能性のある単語の確率分布を調整するパラメータでしたが、`Top K` はモデルが単語を選択する範囲のカットオフを定義します。例えば、`K=50` の場合、モデルは、あるシーケンスで、次に来る可能性の高い 50 の単語から次の単語を選択します。これにより、異常な単語がシーケンスの次に選択される確率が低下します。技術的には、`Top K` は選択する語彙トークンを、確率の上位 K までに限定するトップ K フィルタリングを意味します。これにより、K を小さな値にするほど、モデルは高い確率のトークンの1つを選択するようになります。

**Top P** - `Top P` は、潜在的な選択肢の確率の合計に基づいてカットオフを定義します。`Top P` を 1.0 未満に設定すると、モデルは最も確からしいオプションを考慮し、確率の低いものは無視します。`Top P` は `Top K` に似ていますが、選択肢の数を制限する代わりに、それらの確率の合計に基づいて選択肢を制限します。例えば、「私は...の蹄の音を聞く」というプロンプトの場合、次の単語として「馬」、「シマウマ」、「ユニコーン」をモデルに提供したい場合があります。`Temparature` を最大にして `Top K` や `Top P` を制限しないと、「ユニコーン」のような異常な結果の確率が上がります。`Temparature` を 0 に近づけると、「馬」の確率が上がります。`Temparature` を高い場合でも、 `Top P` を小さな値いに近づけると、「馬」や「シマウマ」の確率が上がり、「ユニコーン」の確率が下がります。

### 長さ

以下のパラメータは、生成される応答の長さを制御します。

**Response length** - 生成される応答で使用するトークンの最小および最大数を設定します。

**Length penalty** - 長さペナルティは、長い応答にペナルティを課すことで、モデルの出力をより簡潔に最適化します。応答長は最小または最大応答長のハードなカットオフですが、長さペナルティはそれとは異なります。技術的には、長さペナルティは長い応答に対してモデルを指数関数的に罰します。0.0 はペナルティがないことを意味します。長いシーケンスを生成するには 0.0 未満の値を、短いシーケンスを生成するには 0.0　より大きな値を設定します。

### 反復

以下のパラメータは、生成された応答での反復を制御するのに役立ちます。

**Repetition penalty (反復ペナルティ)** - 応答で同じ単語(トークン)の反復を防ぎます。1.0 はペナルティがないことを意味します。1.0 を超えると反復が減少します。